## Detect and crop faces to reduce noise for facial recognition

In [ ]:
# installs dedicated module for background removal https://github.com/Ir1d/image-background-remove-tool
%pip install torch pandas opencv-python carvekit --extra-index-url https://download.pytorch.org/whl/cpu

In [2]:
import cv2
import os
import pandas as pd
from PIL import Image

In [3]:
class InputImage:
    def __init__(self, img_name):
        self.img = cv2.imread(img_name)
        self.__name = img_name

    def __str__(self):
        return self.__name

In [ ]:
# download: https://github.com/opencv/opencv_zoo/blob/main/models/face_detection_yunet/face_detection_yunet_2023mar.onnx
detector = cv2.FaceDetectorYN.create("./utils/face_detection_yunet_2023mar.onnx",  "", (0, 0))